In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandapower as pp
from pandapower.networks import case2848rte

from sklearn.preprocessing import StandardScaler    
import joblib

#### Set random seed

In [2]:
seed = 10
random.seed(seed)
np.random.seed(seed)

#### Load power grid

In [3]:
## Load the power grid
net = case2848rte()

## Save the oroiginal graph_structure
pp.to_excel(net, 'graph_structure/original_graph_structure.xlsx')

/Users/yadongzhang/Library/Python/3.9/lib/python/site-packages/pandapower/file_io.py:99: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


#### Set node features

In [4]:
## Create DataFrame to store graph_structure data
graph_structure = pd.DataFrame()

## Set graph_structure index (0-indexed)
graph_structure.index = np.arange(net.bus.shape[0])

In [5]:
## Set voltage for all buses
# Original bus index maybe not sorted
net.bus.sort_index(inplace=True)
# Normalize vn_kv
vn_kv_scaler = StandardScaler()
normalized_vn_kv = vn_kv_scaler.fit_transform(net.bus.vn_kv.to_numpy().reshape((-1, 1))) 
# Original bus index may not range from 0 to net.bus.shape[0]-1
graph_structure['vn_kv'] = normalized_vn_kv    
# Save vn_kv scaler
joblib.dump(vn_kv_scaler, 'graph_structure/vn_kv_scaler.pkl')  

## Set power demand at load buses
# net.load.index = net.load.bus
graph_structure['load_p_mw'] = 0

## Set max and min power generation for all generators
# net.gen.index = net.gen.bus
graph_structure['max_gen_p_mw'] = 0
graph_structure['min_gen_p_mw'] = 0

# Get features from 'ext_grid' (ext_grid bus no: 68)
# net.ext_grid.index = net.ext_grid.bus
# graph_structure['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
# graph_structure['min_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.min_p_mw

## Set power generation cost for all generators
# This needs manual work
poly_cost = pd.read_csv('graph_structure/_poly_cost.csv', index_col=0, header=0)
# net.poly_cost.index = poly_cost_index.values.reshape((-1))
graph_structure['price_1'] = poly_cost.poly_cost1
graph_structure['price_2'] = poly_cost.poly_cost2
# graph_structure['price_2'] = net.poly_cost.cp2_eur_per_mw2

## Set locations for all buses
# Original bus index maybe not sorted
net.bus_geodata.sort_index(inplace=True)
# Normalize geo_data
geo_scaler = StandardScaler()
normalized_geodata = geo_scaler.fit_transform(net.bus_geodata[['x', 'y']])
graph_structure['x_pos'] = normalized_geodata[:, 0]
graph_structure['y_pos'] = normalized_geodata[:, 1]
# Save geo_data scaler
joblib.dump(geo_scaler, 'graph_structure/geo_scaler.pkl')

['graph_structure/geo_scaler.pkl']

#### Get transmission line and transformer index and attributes

In [6]:
## Transmission lines
branch = net.line[['from_bus', 'to_bus']]
branch_attr = net.line[['c_nf_per_km', 'r_ohm_per_km', 'x_ohm_per_km']]

## Transformers
trafo = net.trafo[['hv_bus', 'lv_bus']]
trafo_attr = net.trafo[['sn_mva', 'tap_neutral', 'tap_pos', 'tap_step_percent', 'vk_percent']]

#### Normalize transmission line and transformer attributes

In [7]:
## Normalize transmission line attributes
branch_scaler = StandardScaler()
branch_attr_normalized = branch_scaler.fit_transform(branch_attr)

## Normalize transformer attributes
trafo_scaler = StandardScaler()
trafo_attr_normalized = trafo_scaler.fit_transform(trafo_attr)

#### Save node and edge informartion

In [8]:
## Save graph_structure
graph_structure.to_csv('graph_structure/graph_structure.csv')

## Save branch data
branch.to_csv('graph_structure/branch_index.csv')
branch_attr.to_csv('graph_structure/branch_attr.csv')
pd.DataFrame(branch_attr_normalized).to_csv('graph_structure/branch_attr_normalized.csv')
joblib.dump(branch_scaler, 'graph_structure/branch_attr_scaler.pkl')

## Save trafoormer data
trafo.to_csv('graph_structure/trafo_index.csv')
trafo_attr.to_csv('graph_structure/trafo_attr.csv')
pd.DataFrame(trafo_attr_normalized).to_csv('graph_structure/trafo_attr_normalized.csv')
joblib.dump(trafo_scaler, 'graph_structure/trafo_attr_scaler.pkl')

['graph_structure/trafo_attr_scaler.pkl']